# Simulation of a water box using the $q$-potential

This will simulate a box of SPC/E water using a custom nonbonded force based on a tabulated $q$-potential. It can also perform a simulation using Ewald summation.

Install prerequisites using `conda`:
```bash
    $ conda config --add channels omnia
    $ conda install -c omnia openmm mdtraj packmol
```

In [ ]:
%matplotlib inline
from simtk.openmm.app import *
from simtk.openmm import *
from simtk.unit import *
from sys import stdout
import matplotlib.pyplot as plt
from io import StringIO
import numpy as np, os

In [ ]:
# simulation parameters: more to be added

nsteps  = 2000    # number of MD steps
boxlen  = 18.6206 # size of the cubic box in Å
nwater  = 216     # number of water molecules
NPT     = False   # NPT ensemble using pressure coupling?

# write input file for packmol
PACKMOL_INPUT = """ 
tolerance %f
filetype pdb
output %s

# hoh will be put in a box
# defined by the minimum coordinates x, y and z = 0. 0. 0. and maximum
# coordinates box_size box_size box_size That is, they will be put in a cube of side
# box_size (the keyword "inside cube 0. 0. 0. box_size") could be used as well.

structure %s
  number %d
  inside box 0. 0. 0. %f %f %f 
  add_box_sides 0.0
end structure
""" % (2.,'box.pdb','hoh.pdb', nwater, boxlen, boxlen, boxlen)

!echo '$PACKMOL_INPUT' > packmol_input.txt

In [ ]:
%%writefile hoh.pdb
CRYST1   30.000   30.000   30.000  90.00  90.00  90.00 P 1           1
ATOM      1  O   HOH A   1      27.552  11.051   7.172  1.00  0.00          O
ATOM      2  H1  HOH A   1      27.900  10.721   8.050  1.00  0.00          H
ATOM      3  H2  HOH A   1      26.606  11.355   7.281  1.00  0.00          H 
END

In [ ]:
# use packmol to create a system of randomly placed molecules
!packmol < packmol_input.txt > /dev/null

In [ ]:
def findForce(system, forcetype, add=True):
    """ Finds a specific force in the system force list - added if not found."""
    for force in system.getForces():
        if isinstance(force, forcetype):
            return force
    if add==True:
        system.addForce(forcetype())
        return findForce(system, forcetype)
    return None

def make_qpotential_system(topology, cutoff=0.9*nanometers):
    ff = ForceField('spce-qpot.xml')
    system = ff.createSystem(
        topology, nonbondedMethod=CutoffPeriodic,
        nonbondedCutoff=cutoff, constraints=HBonds, rigidWater=True)

    # tabulated q-potential to be implemented
    x = np.linspace( 0.001,1, 1000 )
    array = 1/x-1
    fwolf = Continuous1DFunction(array, 0*nanometers, 1*nanometers)

    nonbonded = findForce(system, CustomNonbondedForce)
    nonbonded.addTabulatedFunction('wolf', fwolf) # 'wolf(r)' can now be used in energy function
    nonbonded.addGlobalParameter('Rc', cutoff)    # 'Rc' can now be used in energy function
    nonbonded.setEnergyFunction(
        'charge1*charge2*wolf(r)' \
        ' + 4*epsilon*((sigma/r)^12-(sigma/r)^6) ; sigma=0.5*(sigma1+sigma2); epsilon=sqrt(epsilon1*epsilon2)' )
    print('Pair potential:\n  u12(r) =', nonbonded.getEnergyFunction())
    return system

def make_ewald_system(topology, cutoff=0.9*nanometers):
    ff = ForceField('spce.xml')
    return ff.createSystem(
        topology, nonbondedMethod=Ewald,
        nonbondedCutoff=cutoff, constraints=HBonds, rigidWater=True)
    
print('Creating OpenMM System')

pdb = PDBFile('box.pdb')

system = make_qpotential_system( pdb.topology )
#system = make_ewald_system( pdb.topology )

integrator = LangevinIntegrator( 298.15*kelvin, 1.0/picoseconds, 2*femtoseconds )
integrator.setConstraintTolerance(0.00001)

if NPT:
    barostat = MonteCarloBarostat(1.0*bar, 298.15*kelvin, 25) 
    system.addForce(barostat)

platform = Platform.getPlatformByName('CPU')

# Create the Simulation object
sim = Simulation(pdb.topology, system, integrator, platform)
sim.context.setPositions(pdb.positions) # set particle positions

In [ ]:
minimize=True
production=True
restart=True
restartfile='out.chk'

if minimize:
    print('Minimizing energy...')
    sim.reporters.clear()
    sim.minimizeEnergy( tolerance=10*kilojoule/mole, maxIterations=1000 )
    sim.context.setVelocitiesToTemperature( 298.15*kelvin ) # initial random velocities

if production:
    print('Running Production...')
    sim.reporters.clear()
    sim.reporters.append( DCDReporter('out.dcd', 10) )
    sim.reporters.append( StateDataReporter(stdout, 100, step=True, potentialEnergy=True,
                                            temperature=True, density=True, separator='\t',
                                            progress=True,
                                            totalSteps = nsteps) )

    if restart:
        if os.path.isfile( restartfile ):
            with open( restartfile, 'rb') as f:
                print('Loading restart file.')
                sim.context.loadCheckpoint( f.read() )

    sim.step( nsteps )

    with open( restartfile, 'wb') as f:
        print('Saving restart file.')
        f.write( sim.context.createCheckpoint() )

# save final configuration to PDB file
positions = sim.context.getState(getPositions=True).getPositions()
PDBFile.writeFile(sim.topology, positions, open('out.pdb', 'w'))
print('Done!')